In [1]:
import pandas as pd

In [2]:
ad_clicks = pd.read_csv('ad_clicks.csv')
print(ad_clicks.head())

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


In [3]:
#Count how many visitors from each source

source_views = ad_clicks.groupby('utm_source').user_id.count().reset_index()
print(source_views)

  utm_source  user_id
0      email      255
1   facebook      504
2     google      680
3    twitter      215


In [4]:

#If the column ad_click_timestamp is not null, then someone actually clicked on the ad that was displayed.

#Create a new column called is_click, which is True if ad_click_timestamp is not null and False otherwise

ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()
print(ad_clicks)

                                   user_id utm_source            day  \
0     008b7c6c-7272-471e-b90e-930d548bd8d7     google   6 - Saturday   
1     009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook     7 - Sunday   
2     00f5d532-ed58-4570-b6d2-768df5f41aed    twitter    2 - Tuesday   
3     011adc64-0f44-4fd9-a0bb-f1506d2ad439     google    2 - Tuesday   
4     012137e6-7ae7-4649-af68-205b4702169c   facebook     7 - Sunday   
...                                    ...        ...            ...   
1649  fe8b5236-78f6-4192-9da6-a76bba67cfe6    twitter     7 - Sunday   
1650  fed3db6d-8c92-40e3-a4fb-1fb9d7337eb1   facebook     5 - Friday   
1651  ff3a22ff-521c-478c-87ca-7dc7b8f34372    twitter  3 - Wednesday   
1652  ff3af0d6-b092-4c4d-9f2e-2bdd8f7c0732     google     1 - Monday   
1653  ffdfe7ec-0c74-4623-8d90-d95d80f1ba34   facebook   6 - Saturday   

     ad_click_timestamp experimental_group  is_click  
0                  7:18                  A      True  
1                   NaN  

In [5]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()

print(clicks_by_source)

  utm_source  is_click  user_id
0      email     False      175
1      email      True       80
2   facebook     False      324
3   facebook      True      180
4     google     False      441
5     google      True      239
6    twitter     False      149
7    twitter      True       66


In [6]:
#pivot the data based on certain columns

clicks_pivot = clicks_by_source.pivot(
  columns = 'is_click',
  index = 'utm_source',
  values = 'user_id'
).reset_index()


print(clicks_pivot)

is_click utm_source  False  True
0             email    175    80
1          facebook    324   180
2            google    441   239
3           twitter    149    66


In [7]:
#Create a new column in clicks_pivot called percent_clicked which is equal to the percent of users who clicked on the ad from each utm_source.
clicks_pivot['percent_clicked'] = round((clicks_pivot[True] / clicks_pivot[False]) * 100, 2)

print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email    175    80            45.71
1          facebook    324   180            55.56
2            google    441   239            54.20
3           twitter    149    66            44.30


In [8]:
#Count the number of each group if it is equal

print(ad_clicks.groupby(['experimental_group']).user_id.count().reset_index())

  experimental_group  user_id
0                  A      827
1                  B      827


In [9]:
print(ad_clicks.groupby(['is_click', 'experimental_group']).user_id.count().reset_index().pivot(
  columns = 'experimental_group',
  index = 'is_click',
  values = 'user_id'
).reset_index())

experimental_group  is_click    A    B
0                      False  517  572
1                       True  310  255


In [10]:
#Create a dataframe for experiment groups A and B

a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

In [11]:
#For Control A

a_pivot = a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index().pivot(
  columns = 'is_click',
  index = 'day',
  values = 'user_id'
).reset_index()

a_pivot['Percent_Clicked'] = round((a_pivot[True] /a_pivot[False]) * 100, 2)

print(a_pivot)

is_click            day  False  True  Percent_Clicked
0            1 - Monday     70    43            61.43
1           2 - Tuesday     76    43            56.58
2         3 - Wednesday     86    38            44.19
3          4 - Thursday     69    47            68.12
4            5 - Friday     77    51            66.23
5          6 - Saturday     73    45            61.64
6            7 - Sunday     66    43            65.15


In [12]:
#For Control B

b_pivot = b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index().pivot(
  columns = 'is_click',
  index = 'day',
  values = 'user_id'
).reset_index()

b_pivot['Percent_Clicked'] = round((b_pivot[True] /b_pivot[False]) * 100, 2)

print(b_pivot)

is_click            day  False  True  Percent_Clicked
0            1 - Monday     81    32            39.51
1           2 - Tuesday     74    45            60.81
2         3 - Wednesday     89    35            39.33
3          4 - Thursday     87    29            33.33
4            5 - Friday     90    38            42.22
5          6 - Saturday     76    42            55.26
6            7 - Sunday     75    34            45.33
